Importamos las librerias necesarias

In [2]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType
from pyspark.sql import Window
import pyspark.sql.functions as F
import numpy as np

#path del archivo postgresql-42.1.4.jar
dir = "/Users/jasonsolano/Documents/BigData/BigDataTEC/Clase1/DB/"
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", dir+"postgresql-42.1.4.jar") \
    .getOrCreate()

Importamos los datos limpios de postgres

In [3]:
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/ImmoDataBase") \
    .option("user", "postgres") \
    .option("password", "102800") \
    .option("dbtable", "apartmentrentalclean") \
    .load()

df.limit(5).toPandas().head()

,regio1,serviceCharge,heatingType,telekomTvOffer,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,...,typeOfFlat,geo_plz,noRooms,floor,noRoomsRange,garden,livingSpaceRange,regio2,regio3,date
0,Nordrhein_Westfalen,100.00,central_heating,ONE_YEAR_FREE,False,False,29,3.52,2.4,429.00,...,ground_floor,45329,2.0,0.0,2,False,2,Essen,Karnap,May19
1,Nordrhein_Westfalen,84.00,central_heating,ONE_YEAR_FREE,False,True,4,4.92,2.4,578.00,...,roof_storey,48282,3.0,2.0,3,False,3,Steinfurt_Kreis,Emsdetten,Sep18
2,Nordrhein_Westfalen,115.00,night_storage_heater,ONE_YEAR_FREE,False,False,11,4.69,40.0,365.00,...,roof_storey,46242,3.0,3.0,3,False,2,Bottrop,Lehmkuhle,Sep18
3,Sachsen_Anhalt,33.00,self_contained_central_heating,ONE_YEAR_FREE,False,False,9,0.93,40.0,358.00,...,roof_storey,39218,2.0,0.0,2,False,2,Salzlandkreis,Schönebeck_Elbe,May19
4,Sachsen,169.75,gas_heating,ONE_YEAR_FREE,False,True,9,-1.89,2.4,449.49,...,apartment,9126,2.0,2.0,2,False,2,Chemnitz,Bernsdorf,May19


Se imprime el esquema

In [4]:
df.printSchema()

root
 |-- regio1: string (nullable = true)
 |-- serviceCharge: double (nullable = true)
 |-- heatingType: string (nullable = true)
 |-- telekomTvOffer: string (nullable = true)
 |-- newlyConst: boolean (nullable = true)
 |-- balcony: boolean (nullable = true)
 |-- picturecount: integer (nullable = true)
 |-- pricetrend: double (nullable = true)
 |-- telekomUploadSpeed: double (nullable = true)
 |-- totalRent: double (nullable = true)
 |-- firingTypes: string (nullable = true)
 |-- hasKitchen: boolean (nullable = true)
 |-- geo_bln: string (nullable = true)
 |-- cellar: boolean (nullable = true)
 |-- yearConstructedRange: double (nullable = true)
 |-- baseRent: double (nullable = true)
 |-- livingSpace: double (nullable = true)
 |-- geo_krs: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- lift: boolean (nullable = true)
 |-- baseRentRange: integer (nullable = true)
 |-- typeOfFlat: string (nullable = true)
 |-- geo_plz: integer (nullable = true)
 |-- noRooms: doub

In [7]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

for i in ["serviceCharge"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df = pipeline.fit(df).transform(df).withColumn(i, unlist(i+"_Scaled")).drop(i+"_Vect")

df.select("serviceCharge").show()

+-------------+
|serviceCharge|
+-------------+
|        0.001|
|        0.001|
|        0.001|
|          0.0|
|        0.001|
|        0.001|
|          0.0|
|        0.001|
|        0.001|
|        0.002|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
|        0.001|
+-------------+
only showing top 20 rows



In [9]:
datasetSize = df.count()

df.groupBy('serviceCharge').count().sort(col("count").desc()).withColumn('perc_of_count_total', (F.col('count') / datasetSize)) \
  .toPandas().head(20)

,serviceCharge,count,perc_of_count_total
0,0.001,119653,0.709648
1,0.002,22993,0.136369
2,0.000,22417,0.132953
3,0.003,2757,0.016351
4,0.004,481,0.002853
5,0.005,168,0.000996
6,0.006,40,0.000237
7,0.007,40,0.000237
8,0.008,16,0.000095
9,0.009,12,0.000071
